In [ ]:
# pip install pycryptodome

In [ ]:
# pip install opencv-python

In [3]:
#importing the necessary libraries
import os
import sys
import cv2
import numpy as np
import shutil
from Crypto.Cipher import AES

In [4]:
def load_frames(mode = "train"):
    """
    Extract frames from the video and save them as JPEG images in a folder
    Args:
        mode (str): either "train" or "test", determines which video to load
    Returns:
        numpy.ndarray: an array containing all of the frames from the video
    """
    # Checking if directory exists, if it does remove it
    if os.path.isdir("./data"):
        shutil.rmtree("./data")

    # Creating a directory to store frames
    dir_path = os.path.join("./data", mode+'_imgs')
    os.makedirs(dir_path, exist_ok=True)

    # Opening the video file
    video_path = 'assets/train.mp4'
    vidcap = cv2.VideoCapture(video_path)

    # Extracting frames from the video and save as JPEG files
    success,image = vidcap.read()
    count = 0
    while success:
        cv2.imwrite("%s/%d.jpg" % (dir_path,count), image)     # save frame as JPEG file      
        success,image = vidcap.read()
        count += 1
        if count % 300 == 0:
            print("%d frames read" % count)

    # Loading the saved JPEG files into a numpy array
    frame_cnt = len(os.listdir(dir_path))
    vid_frames = np.empty((frame_cnt,1080,1920,3),dtype='uint8')
    for i in range(0,frame_cnt):
        frame = cv2.imread(dir_path + '/' + str(i) + ".jpg")
        vid_frames[i] = frame
        sys.stdout.write("\rLoading frame " + str(i))

    return vid_frames

# Calling the function to extract frames from the "train" video
load_frames()


300 frames read
600 frames read
Loading frame 744

array([[[[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        [[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        [[138, 102,  54],
         [138, 102,  54],
         [138, 102,  54],
         ...,
         [136,  97,  65],
         [136,  97,  65],
         [136,  97,  65]],

        ...,

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
         [  4,   0,   0],
         [  4,   0,   0]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
         [  4,   0,   0],
         [  4,   0,   0]],

        [[  1,   1,   1],
         [  1,   1,   1],
         [  1,   1,   1],
         ...,
         [  4,   0,   0],
        

In [ ]:
video_path = 'assets/train.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize the previous frame and previous points for optical flow
prev_frame = None
prev_pts = None

# Initialize a font for displaying the speed value
font = cv2.FONT_HERSHEY_SIMPLEX

# Set the reference distance in meters and the reference time in seconds
ref_distance = 10  # 10 meters
ref_time = 3  # 3 seconds

# Initialize an empty list to store the speed values
speeds = []

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # If this is not the first frame, compute optical flow
        if prev_frame is not None:
            # Compute optical flow using the previous frame and the current frame
            flow = cv2.calcOpticalFlowFarneback(prev_frame, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
            
            # Compute the magnitude and angle of the optical flow vectors
            mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
            
            # Convert the angle to hue value
            hsv = np.zeros_like(frame)
            hsv[...,0] = ang * 180 / np.pi / 2
            
            # Set the value and saturation to maximum
            hsv[...,1] = 255
            hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            
            # Convert the HSV image to BGR for display
            bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            
            # Overlay the optical flow vectors on top of the frame
            frame = cv2.addWeighted(frame, 1, bgr, 2, 0)
            
            # Compute the speed of the car based on the mean magnitude of optical flow vectors
            speed = np.mean(mag) * ref_distance / ref_time
            
            # Convert the speed value from meters per second to kilometers per hour
            speed_kmph = speed * 3.6
            
            # Draw the speed value on top of the frame
            cv2.putText(frame, 'Speed: {:.2f} km/h'.format(speed_kmph), (10, 50), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Display the frame with optical flow vectors and speed value
            cv2.imshow('frame', frame)
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
            
            # Append the speed value to the list of speeds
            speeds.append(speed_kmph)
        
        # Update the previous frame and previous points for optical flow
        prev_frame = gray.copy()
        prev_pts = cv2.goodFeaturesToTrack(prev_frame, 200, 0.01, 10)
    else:
        break

cap.release()
cv2.destroyAllWindows()

# Convert the list of speeds to a NumPy array
speeds = np.array(speeds)

# Print the mean and standard deviation of the speeds
print('Mean speed: {:.2f} km/h'.format(np.mean(speeds)))
print('Std dev of speed: {:.2f} km/h'.format(np.std(speeds)))

1   HIToolbox                           0x00000001ab5105c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001ab510188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x00000001ab4b3310 SetMenuBarObscured + 372
4   HIToolbox                           0x00000001ab4b2ee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
5   HIToolbox                           0x00000001ab4acfcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
6   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x00000001ab4ac96c AcquireEventFromQueue + 432
8   HIToolbox                           0x00000001ab49be0c ReceiveNextEventCommon + 712
9   HIToolbox                           0x00000001ab49bb2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x00000001a504184c _DPS

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

...

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab636d4c _ZN15MenuBarInstance21IsAutoShowHideAllowedEv + 284
2   HIToolbox                           0x00000001ab510230 _ZN15MenuBarInstance24UpdateAutoShowVisibilityE5Pointh + 40
3   HIToolbox                           0x00000001ab47d30c _ZN15MenuBarInstance16ForEachMenuBarDoEU13block_pointerFvPS_E + 72
4   HIToolbox                           0x00000001ab5108a0 _ZN15MenuBarInstance20AutoShowHideObserverEjP14OpaqueEventRefPv + 216
5   HIToolbox                           0x00000001ab473cd0 _NotifyEventLoopObservers + 176
6   HIToolbox                           0x00000001ab4a7bb0 PostEventToQueueInternal + 696
7   HIToolbox                           0x00000001ab4a9340 _ZL29CreateAndPostEventWithCGEventP9__CGEventjhP17__CFMachPortBoost + 460
8   HIToolbox                           0x00000001ab4a8c44 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 296
9   HIToolbox                           0x00000001ab4a8ae0 _ZL14Messag

1   HIToolbox                           0x00000001ab49b90c _ZN15MenuBarInstance22RemoveAutoShowObserverEv + 44
2   HIToolbox                           0x00000001ab4da478 _ZL17BroadcastInternaljPvh + 184
3   SkyLight                            0x00000001a69ed214 _ZN12_GLOBAL__N_123notify_datagram_handlerEj15CGSDatagramTypePvmS1_ + 896
4   SkyLight                            0x00000001a69ebe10 CGSSnarfAndDispatchDatagrams + 808
5   SkyLight                            0x00000001a6d13a9c SLSGetNextEventRecordInternal + 344
6   SkyLight                            0x00000001a6b14fb0 SLEventCreateNextEvent + 16
7   HIToolbox                           0x00000001ab4a8b58 _ZL38PullEventsFromWindowServerOnConnectionjhP17__CFMachPortBoost + 60
8   HIToolbox                           0x00000001ab4a8ae0 _ZL14MessageHandlerP12__CFMachPortPvlS1_ + 60
9   CoreFoundation                      0x00000001a1ded410 __CFMachPortPerform + 260
10  CoreFoundation                      0x00000001a1dbdf98 __CFRUNLO

In [11]:
# Exporting the speeds array to a file
np.save('speeds.npy', speeds)

In [13]:
def encrypt_file_AES(key, in_filename, out_filename=None, chunksize=64*1024):
    """Encrypts a file using AES encryption.

    Args:
        key (bytes): The encryption key. Must be 16, 24, or 32 bytes long.
        in_filename (str): The name of the input file.
        out_filename (str): The name of the output file. If not specified, 
            will be set to the input file name with '.enc' extension.
        chunksize (int): The size of each chunk in bytes.

    Returns:
        str: The name of the output file.
    """
    # Set the output file name if not specified
    if not out_filename:
        out_filename = in_filename + '.enc'

    # Generate a random IV
    iv = os.urandom(16)

    # Create an AES cipher object
    cipher = AES.new(key, AES.MODE_CBC, iv)

    # Open the input and output files
    with open(in_filename, 'rb') as infile, open(out_filename, 'wb') as outfile:
        # Write the IV to the output file
        outfile.write(iv)

        # Encrypt each chunk of the input file and write it to the output file
        while True:
            chunk = infile.read(chunksize)
            if len(chunk) == 0:
                break
            elif len(chunk) % 16 != 0:
                # Pad the chunk to a multiple of 16 bytes
                chunk += b' ' * (16 - len(chunk) % 16)
            encrypted_chunk = cipher.encrypt(chunk)
            outfile.write(encrypted_chunk)

    return out_filename


In [14]:
# Convert the key string to bytes and truncate it to 16 bytes
key = 'this_is_a_16_byte_key'.encode('utf-8')[:16]

# Encrypt the 'speeds.npy' file using AES encryption
encrypted_file = encrypt_file_AES(key, 'speeds.npy')


In [15]:
def decrypt_file_AES(key, in_filename, out_filename=None, chunksize=24*1024):
    """Decrypts a file that has been encrypted using AES encryption.

    Args:
        key (bytes): The encryption key. Must be 16, 24, or 32 bytes long.
        in_filename (str): The name of the input file.
        out_filename (str): The name of the output file. If not specified, 
            will be set to the input file name with '.dec' extension.
        chunksize (int): The size of each chunk in bytes.

    Returns:
        str: The name of the output file.
    """
    # Set the output file name if not specified
    if not out_filename:
        out_filename = in_filename + '.dec'

    # Open the input and output files
    with open(in_filename, 'rb') as infile, open(out_filename, 'wb') as outfile:
        # Read the IV from the input file
        iv = infile.read(16)

        # Create an AES cipher object
        cipher = AES.new(key, AES.MODE_CBC, iv)

        # Decrypt each chunk of the input file and write it to the output file
        while True:
            chunk = infile.read(chunksize)
            if len(chunk) == 0:
                break
            decrypted_chunk = cipher.decrypt(chunk)
            outfile.write(decrypted_chunk)

    return out_filename

In [22]:
# Load the key from a file or use the same key that was used for encryption
key = 'this_is_a_16_byte_key'.encode('utf-8')[:16]

# Decrypt the encrypted 'speeds.npy' file using AES decryption
decrypted_file = decrypt_file_AES(key, 'speeds.npy.enc')

In [24]:
# Loading the original 'speeds.npy' file
speeds_orig = np.load('speeds.npy')

# Loading the decrypted 'speeds.npy.enc.dec' file
speeds_dec = np.load('speeds.npy.enc.dec')

# Verifing that the arrays are equal
if np.array_equal(speeds_orig, speeds_dec):
    print("The arrays are equal. The decryption was completed successfully")
else:
    print("The arrays are not equal.")

The arrays are equal. The decryption was completed successfully
